# Описание проекта

   Сеть фитнес-центров «Культурист-датасаентист» разрабатывает стратегию взаимодействия с клиентами на основе аналитических данных. 
   
   Распространённая проблема фитнес-клубов и других сервисов — отток клиентов. Как понять, что клиент больше не с вами? Индикаторы оттока зависят от специфики отрасли. Для фитнес-центра можно считать, что клиент попал в отток, если за последний месяц ни разу не посетил спортзал. Конечно, не исключено, что он уехал в отпуск и по приезде обязательно продолжит ходить на фитнес. Однако чаще бывает наоборот. Если клиент начал новую жизнь с понедельника, немного походил в спортзал, а потом пропал — скорее всего, он не вернётся. 
   
   Чтобы бороться с оттоком, отдел по работе с клиентами «Культуриста-датасаентиста» перевёл в электронный вид множество клиентских анкет. Наша задача — провести анализ и подготовить план действий по удержанию клиентов.  
   
   А именно: 
    
- научиться прогнозировать вероятность оттока (на уровне следующего месяца) для каждого клиента;

- сформировать типичные портреты клиентов: выделить несколько наиболее ярких групп и охарактеризовать их основные свойства;

- проанализировать основные признаки, наиболее сильно влияющие на отток;

- сформулировать основные выводы и разработать рекомендации по повышению качества работы с клиентами:

  - выделить целевые группы клиентов;

  - предложить меры по снижению оттока;

  - определить другие особенности взаимодействия с клиентами.

# Описание данных

«Культурист-датасаентист» предоставил сведения в csv-файлах. Заказчик подготовил данные, которые содержат данные на месяц до оттока и факт оттока на определённый месяц. Набор данных включает следующие поля:

- Данные клиента за предыдущий до проверки факта оттока месяц:

  - 'gender' — пол;

  - 'Near_Location' — проживание или работа в районе, где находится фитнес-центр;

  - 'Partner' — сотрудник компании-партнёра клуба (сотрудничество с компаниями, чьи сотрудники могут получать скидки на абонемент — в таком случае фитнес-центр хранит информацию о работодателе клиента);

  - Promo_friends — факт первоначальной записи в рамках акции «приведи друга» (использовал промо-код от знакомого при оплате первого абонемента);

  - 'Phone' — наличие контактного телефона;

  - 'Age' — возраст;

  - 'Lifetime' — время с момента первого обращения в фитнес-центр (в месяцах).

- Информация на основе журнала посещений, покупок и информация о текущем статусе абонемента клиента:

  - 'Contract_period' — длительность текущего действующего абонемента (месяц, 6 месяцев, год);

  - 'Month_to_end_contract' — срок до окончания текущего действующего абонемента (в месяцах);

  - 'Group_visits' — факт посещения групповых занятий;

  - 'Avg_class_frequency_total' — средняя частота посещений в неделю за все время с начала действия абонемента;

  - 'Avg_class_frequency_current_month' — средняя частота посещений в неделю за предыдущий месяц;

  - 'Avg_additional_charges_total' — суммарная выручка от других услуг фитнес-центра: кафе, спорттовары, косметический и массажный салон.

  - 'Churn' — факт оттока в текущем месяце.

## Загрузите данные

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import transforms
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('/datasets/gym_churn.csv')

In [ ]:
df.sample(10)

## Проведите исследовательский анализ данных (EDA)

Скорректируем названия столбцов и проверим датасет на пропуски

In [ ]:
df.columns = df.columns.str.lower()
print('Кол-во дублей в строках: {}\n'.format(df.duplicated().sum()))
print(df.info())
print('\n Пропусков в данных не обнаружено.')
df.head(5)

Изменим формат поля "month_to_end_contract" с плавающей точкой на целочисленный

In [ ]:
df['month_to_end_contract'] = df['month_to_end_contract'].astype(np.int64)
df.info()

Провели первичный анализ датасета, исправили некоторые моменты, которые могли бы помешать более детальному анализу. Датасет готов к работе

Теперь посмотрим на средние значения признаков в двух группах — тех, кто ушел в отток и тех, кто остался

In [ ]:
df.groupby('churn').agg({'mean', 'std', 'var'}).T

**Промежуточные выводы:**

- примерно одинаковое количество мужчин и женщин;
- средний возраст посетителей в районе 27-30 лет;
- большинство клиентов проживают или работают в районе, где находится фитнес-центр;
- примерно половина клиентов являются сотрудниками компании-партнёра клуба;
- факт первоначальной записи в рамках акции «приведи друга» около 30-35 %;
- в среднем клиенты берут абонемент на 6 мес., но высокий показатель стандартного отклонения коэффициента вариативности говорит о том, что абонемент на 6 мес. не самый часто покупаемый;

Посомотрим на столбчатые гистограммы и распределения признаков для тех, кто ушёл (отток) и тех, кто остался (не попали в отток)

In [ ]:
for column in df.columns:
    plt.figure(figsize=(5, 5))
    df.groupby('churn')[column]
    sns.distplot(df[column])
    plt.title(column)
    plt.show()

In [ ]:
columns_d1 = ['avg_additional_charges_total', 'avg_class_frequency_total', 
                    'avg_class_frequency_current_month', 'lifetime', 'age', 
                    'contract_period', 'month_to_end_contract']

In [ ]:
for column in df.columns:
  if column not in columns_d1:
    df.groupby('churn')[column].hist()
    plt.title(column)
    plt.legend(['Оставшиеся', 'Отток'])
    plt.show()

**Промежуточные выводы:**

- фактор расстояния не является основной причиной, т.к. видно что даже клиенты, находящиеся рядом с фитнес-центром тоже активно уходят;
- в основном приобретают месячный абонемент;
- большинство клиентов не посещают групповые занятия(Что вполне нормально. Я вот тоже хожу заниматься в основном в соло);
- основное распределение трат в диапазоне от 0 до 150, далее показатель снижается и совсем редко превышает 400;
- распределение продолжительности посещения в основном сосредоточено на отметке в 1-2 месяца, далее показатель резко падает.

Чтож, посмотрим на матрицу корреляций

In [ ]:
cm=df.corr()
plt.figure(figsize=(10,7))
sns.heatmap(cm,annot=True,fmt='.2f',vmin=-0.5,annot_kws={"size": 10}, cmap="Blues")
plt.title('Матрица корреляций', fontsize = 12)
plt.show()

**Вывод:**

Видим, что признаками с сильной корреляцией являются:

- средняя частота посещений в неделю за предыдущий месяц и суммарная выручка от других услуг фитнес-центра: кафе, спорт-товары, косметический и массажный салон
- срок до окончания текущего действующего абонемента (в месяцах) и длительность текущего действующего абонемента (месяц, 3 месяца, 6 месяцев, год);

В общем то они показывают одно и тоже и для будущей модели они скорее не нужны, чем полезны.

## Построим модель прогнозирования оттока клиентов

Разобьём данные на обучающую и валидационную выборку, после чего обучим модель на train-выборке двумя способами:
- логистической регрессией,
- случайным лесом.

А также оценим метрики accuracy, precision и recall для обеих моделей на валидационной выборке и сравним по ним модели. 

In [ ]:
X = df.drop(['churn'], axis = 1)
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
scaler = StandardScaler()
X_train_st = scaler.fit_transform(X_train)
X_test_st = scaler.transform(X_test)

lr_model = LogisticRegression(random_state=0)
lr_model.fit(X_train_st, y_train)
lr_predictions = lr_model.predict(X_test_st)
lr_probabilities = lr_model.predict_proba(X_test_st)[:,1]
print('Метрики для модели логистической регрессии:')
print('accuracy_score: {}\nprecision_score: {}\nrecall_score:{}'.format(
    accuracy_score(y_test, lr_predictions),
    precision_score(y_test, lr_predictions),
    recall_score(y_test, lr_predictions)
))

rf_model = RandomForestClassifier(n_estimators = 100, random_state = 0) 
rf_model.fit(X_train_st, y_train)
rf_predictions = rf_model.predict(X_test_st)
rf_probabilities = rf_model.predict_proba(X_test_st)[:,1]
print('\n Метрики для модели случайного леса:')
print('accuracy_score: {}\nprecision_score: {}\nrecall_score:{}'.format(
    accuracy_score(y_test, rf_predictions),
    precision_score(y_test, rf_predictions),
    recall_score(y_test, rf_predictions)
))

features = pd.DataFrame(lr_model.coef_.T, X.columns).reset_index()
features.columns = ['feature', 'coef']
features['coef'] = features['coef'].apply(lambda x: abs(x))
features = features.sort_values(by='coef', ascending=False)
print('\n Коэффициенты признаков в оптимальной функции логистической регрессии:')
print(features)

**Вывод: Доля правильности и полнота прогнозов чуть выше в модели *логистической регрессии*, следовательно на данном этапе эта модель показала себя лучше**

## Сделаем кластеризацию клиентов

Стандартизируем данные, построим матрицу расстояний и нарисуем дендрограмму

In [ ]:
df_sc = scaler.fit_transform(df.drop(columns = ['churn'])) 

In [ ]:
linked = linkage(df_sc, method = 'ward')
plt.figure(figsize=(23, 7))  
dendrogram(linked, orientation='top', no_labels=True)
plt.title('Дендрограмма иерархической кластеризации клиентов фитнес-клуба без `churn`', size=14)
plt.show()

Четыре разных цвета на дендограмме, следовательно оптимальное число кластеров - 4. Однако в соответсвии с ТЗ - число кластеров должно равняться пяти.

Т.к. четвертый кластер очень большой, попробуем добавить столбец churn и посмотрим на результат

In [ ]:
df_sc_1 = scaler.fit_transform(df) 
linked = linkage(df_sc, method = 'ward')
plt.figure(figsize=(23, 7))  
dendrogram(linked, orientation='top', no_labels=True)
plt.title('Дендрограмма иерархической кластеризации клиентов фитнеса (с `churn`)', size=14)
plt.show()

**Промежуточный вывод:**

На новой дендрограмме число кластеров соотвествует 5, и в целом данный график теперь выглядит более корректно

Теперь обучим модель кластеризации на основании алгоритма K-Means и спрогнозируем кластеры клиентов. Сначала зададим число кластеров равное 5, применим алгоритм к данным и сформируем вектор кластеров. После чего сохраним метки кластера в поле *cluster_km*

In [ ]:
km = KMeans(n_clusters = 5, random_state = 0)
labels = km.fit_predict(df_sc_1)
df['cluster_km'] = labels

Посмотрим на средние значения признаков для кластеров

In [ ]:
df.groupby('cluster_km').mean().T

**Промежуточный вывод:**

1. Почти все клиенты в оттоке попали в кластер № 2:

- представителей мужчин и женщин в нем примерно поровну;
- средний возраст клиентов находится в районе 27 лет;
- почти все признаки ниже чем в остальных кластерах, особенно это заметно в таких показателях, как: длительность текущего действующего абонемента, суммарная выручка от других услуг фитнес-центра и длительность текущего действующего абонемента;

2. В кластере № 0 из оттока 45 % клиентов. Отличительной особенностью этого кластера является полное отсутствие в базе информации о проживании или работе в районе, где находится фитнес-центр.

3. Самые надежные кластеры № 1 и 3. Их отличительными особенностями являются:

- самые возрастные клиенты - в обоих кластерах средний возраст около 30 лет;
- стабильно высокая выручка от других услуг фитнес-центра;

4. Отдельно можно выделить, что в кластере № 1 самая высокая средняя частота посещений.

Теперь построим распределения признаков для кластеров

In [ ]:
for value in [0, 1, 2, 3, 4]:
  df[df['cluster_km']==value].hist(figsize=(17,10))
  plt.suptitle('\nРаспределение признаков для кластера {}'.format(value), fontsize = 15)
  plt.show()

**Промежуточный вывод:**

Из ранее сделанных выводов по клиентам из оттока дополнительно можно сказать:

- около 75 % всех клиентов живут или работают рядом с фитнес-центром;
- половина клиентов не являются сотрудниками компании-партнера;
- почти треть клиентов пришли по акции "приведи друга".

In [ ]:
df.groupby('cluster_km').agg({'churn':'mean'})

**Промежуточный вывод:**

Как уже было сказано ранее, кластер № 2 почти полностью сосостоит из клентов находящихся в оттоке, а клиенты из кластеров № 1 и 3 являются наиболее лояльными.

# Итоговые выводы и рекомендации:

В ходе выполнения проекта были проведены:
- исследовательский анализ данных; 
- построение двух моделей прогнозирования оттока клиентов;
- кластеризация клиентов для определения поведенческих групп клиентов и отличительных признаков клиентов из имеющих наибольшую долю оттока.

По ходу исследования, в отношении картины в целом, мы пришли к следующим выводам:

- Распределение по полу примерно одинаково
- Большинство клиентов живет или работает рядом с фитнес-клубом
- Примерно половина клиентов являются сотрудниками партнерских организаций
- Около 30% клиентов пришли по программе "Приведи друга"
- В базе находятся мобильные номера примерно 90% клиентов
- Средняя длительность абонементов клиентов - 4-5 месяцев

Что касается отличий признаков у клиентов, попавших в отток и оставшихся, можно сказать:

- Ближе живущие или работающие клиенты реже попадают в отток, что вполне логично.
- Сотрудники компаний-партнеров реже перестают посещать фитнесс-клуб чем остальные клиенты.
- Наличие мобильного номера клиента не отличается в группах.
- Средняя длительность договора больше у оставшихся пользователей, в то время как у большей части клиентов в оттоке срок договора составляет 1 месяц (скорее это говорит о том, что чаще клиенты заключают договор на 1 месяц).
- Оставшиеся пользователи чаще ходят на групповые занятия.
- У групп есть разница в среднем возрасте клиентов.
- Так же оставшиеся клиенты чаще пользуются другими услугами фитнес-центра.

**Рекомендуемые меры по удержанию клиентов:**  

1. Сделать более привлекательным к приобретению новым клиентам абонементы на срок от 6 месяцев;
2. Провести промо-акции или как то ещё увеличить интерес к групповым занятиям;  
3. Проанализировать дополнительные товары и услуги с более высоким спросом, чтобы в дальнейшем использовать их для привлечения клиентов;  